In [1]:
# Импорт библиотек
import os
import time
import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 10
Maxiterations_cotel = 5
Maxiterations_turbine = 15

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 20,
    "kolichestvo": 8,
    "Visota": 20,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}
PKM_zaryad = True
PKM_razryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 10
pkm_pgu_tol = 10**-2

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]


########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расхода пара на ПКМ из условия заполнения хранилища################
########РАСЧЕТ мощности ГТУ из условия тепловой мощности и ################

################Расчет минимальной нагрузки ГТУ при остальных нормальных условиях############

Max_iterations_minimum = 10

n_GTU_it = []
if Сalculate_minimum == True:
    gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    for i in range(Max_iterations_minimum):
        n_GTU_it.append(round(n_GTU,3))
        print("n_GTU:", n_GTU_it)
        if i < 6:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (2, 2, 15, 3)
        else:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
                Iter_pkm,
            )

        Delt_Gcnd = 100
        Delt_Nturb = 100
        Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
        Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")
        n_GTU = n_GTU - Delta_min / 5
        GTU_input.at["n", 1] = n_GTU
        arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
            New_iterations_KU_TU,
            New_iterations_cotel,
            New_iterations_turbine,
        )
        gas_streams = Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)
        print(
            f"Время {i+1} итерации расчета мощности ГТУ при ПГУ с ПКМ: --- {round((time.time() - start_time), 1)} сек. ---"
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")
        if abs(Delta_min) < Calctolerance:
            arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
            Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
        )
            gas_streams = Calculate_CCGT_PKM_iter(
                arguments_all_it, New_Iter_pkm, pkm_pgu_tol
            )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            print("n_GTU_it", n_GTU_it)
            break
        if i == Max_iterations_minimum - 1:
            print(
                "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
                i + 1,
            )
            print("n_GTU_it", n_GTU_it)

n_GTU: [1]
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 48.8
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  10.0 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 2 итерации расчета КУ+ТУ:---  22.3 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 2
Pnd_it [0.51016, 0.47684]
Pvd_it [5.35815, 4.17631]
fin КУ и ТУ:--- 22.3 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ: --- 24.5 сек. ---
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 113.7
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  8.0 сек. ---
Давление на 

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,527.600273,0.1,941.958209,386.442692,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,52.690336,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,563.571217,0.1,1012.650521,439.133027,0.771725,0.109764,0.037506,0.074242,0.006763
PEVD-IVD,484.628392,0.1,921.186874,439.133027,0.771725,0.109764,0.037506,0.074242,0.006763
IVD-EVD,266.153719,0.1,676.782425,439.133027,0.771725,0.109764,0.037506,0.074242,0.006763
EVD-PPND,203.497575,0.1,608.923990,439.133027,0.771725,0.109764,0.037506,0.074242,0.006763
PPND-IND,201.128104,0.1,606.374738,439.133027,0.771725,0.109764,0.037506,0.074242,0.006763
IND-GPK,149.504823,0.1,551.109304,439.133027,0.771725,0.109764,0.037506,0.074242,0.006763
GPK-out,98.448240,0.1,496.904847,439.133027,0.771725,0.109764,0.037506,0.074242,0.006763


In [3]:
water_streams

,T,P,H,G,S,X
AIR,-29.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,497.818304,3.757641,3443.805864,61.663494,NaN,NaN
IVD-PEVD,256.501442,4.431379,2798.396372,61.663494,NaN,NaN
EVD-IVD,247.761963,4.431379,1074.897502,61.663494,NaN,NaN
PEN-EVD,141.067618,4.431379,596.367194,61.663494,NaN,NaN
BND-PEN,139.871661,0.360228,588.611236,61.663494,NaN,NaN
PPND-DROSND,185.310027,0.385820,2832.534592,11.358732,NaN,NaN
IND-PPND,139.871660,0.360228,2733.276873,11.358732,NaN,NaN
GPK-IND,139.871661,0.360228,593.181440,73.022226,NaN,NaN
GPK-REC,139.871661,0.360228,593.181440,72.829196,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3687.984095,45.441807,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3030.242194,45.441807,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.0,2.0,2555.053441,45.441807,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.0,2.0,791.135714,45.441807,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.0,2.0,1348.041621,18.074747,0.0,0.0,0.168668,0.0,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separacc-Sepout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-GTU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
electric

,Ni,N,KPDm,KPD
PEN,0.478260,0.618976,0.772662,0.566141
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,102.434750,NaN,29.549610
DK,NaN,6.167231,NaN,NaN
Turbine,17.531001,NaN,NaN,NaN
Tots1,10.024629,NaN,NaN,0.831135
Tots2,1.854166,NaN,NaN,0.812366
Tots3,0.463828,NaN,NaN,0.754431
Tots4,5.188377,NaN,NaN,0.573060


In [6]:
heaters

,Qw,Qg,KPD
PEVD,39763.061020,40164.708101,0.99
IVD,106252.804925,107326.065581,0.99
EVD,29500.891417,29798.880219,0.99
PPND,1108.266277,1119.460886,0.99
IND,24026.188549,24268.877322,0.99
GPK,23564.937334,23802.967004,0.99
SP2,1.536204,NaN,0.99
SP1,22.866750,NaN,0.99
OD,1.701616,NaN,0.99
Strans,129818.488861,NaN,NaN


In [7]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,NaN,250.0,10000.0
